# Utility Methods

In [1]:
from IPython import get_ipython
import ast
from ast import *
import inspect
from typing import Any
import re

def source_cell(cell_id):
  ipython = get_ipython()
  cells = ipython.history_manager.input_hist_raw
  pattern = re.compile(rf"^\s*#\s*code:\s*{cell_id}\s*$", re.IGNORECASE)

  for index, cell in enumerate(cells):
    if cell:
      first_line = cell.strip().splitlines()[0]
      if pattern.match(first_line):
          return cell

  return None

def ast_cls_assert(node_A,node_B):
  print("Result:")
  print(ast.unparse(node_A))
  print("Expected:")
  print(ast.unparse(node_B))
  assert ast.dump(node_A) == ast.dump(node_B), "Assertion Failed"
  print("Assertion passed")


import ast
from ast import *
import inspect
# it assume that both are function def
def ast_fun_assert(node_A,node_B):
  a = node_A.body[0]
  a.name='anom'
  b = node_B.body[0]
  b.name='anom'
  assert ast.dump(a) == ast.dump(b), "Assertion Failed"
  print("Assertion passed")

# Ejercicio 1: bad type checking

Se considera que el siguiente código en python:

```
x = 2
if type(x) == int:
    print("Integer")
```
Cree un transformador que auto-repare el código anterior utilizando la función isinstance en python

```
if isinstance(x,int):
    print("Integer")
```

## Test Code

In [2]:
def example_before(x):
  if type(x) == int:
    print("Integer")

def example_after(x):
  if isinstance(x,int):
    print("Integer")

## Complete Transformer


In [3]:
class CheckTypeTransformer(NodeTransformer):
  pass

## Test

In [4]:
# Arrange
before_ast =  ast.parse(inspect.getsource(example_before))
expected_ast =  ast.parse(inspect.getsource(example_after))
# Act
transformer = CheckTypeTransformer()
after_ast = transformer.visit(before_ast)
ast.fix_missing_locations(after_ast)
# Assert
ast_fun_assert(after_ast, expected_ast)

AssertionError: Assertion Failed

# Ejercicio 2: If True Transformer
Se considera que el siguiente código en python:

```python
if a > b:
    return True
  else:
    return False
```
Cree un transformador que auto-repare el código anterior eliminando el if innecesario:

```python
return a > b
```


## Test Code

In [5]:
def if_before(a,b):
  if a > b:
    return True
  else:
    return False

def if_after(a,b):
  return a > b

## Complete Transformer

In [6]:
import ast
from ast import *
import inspect
from typing import Any

class IfTransformer(ast.NodeTransformer):
    pass


## Test

In [7]:
# Arrange
before_ast =  ast.parse(inspect.getsource(if_before))
expected_ast =  ast.parse(inspect.getsource(if_after))
# Act
transformer = IfTransformer()
after_ast = transformer.visit(before_ast)
ast.fix_missing_locations(after_ast)
# Assert
ast_fun_assert(after_ast, expected_ast)

AssertionError: Assertion Failed

# Ejercicio 3: Inject a Mutant, replace + by -

Se considera que el siguiente código en python:

```
a + b + c - a - b * 2
```
Cree un transformador que injecte un mutante al código, reemplazando los + por menos

```
a - b - c - a - b * 2
```

## Test Code

In [8]:
def mutant_before(a,b,c):
  return a + b + c - a - b * 2

def mutant_after(a,b,c):
  return a - b - c - a - b * 2

## Complete Trasnformer


In [9]:
class AddToSubTransformer(ast.NodeTransformer):
   pass

## Test

In [10]:
# Arrange
before_ast =  ast.parse(inspect.getsource(mutant_before))
expected_ast =  ast.parse(inspect.getsource(mutant_after))
# Act
transformer = AddToSubTransformer()
after_ast = transformer.visit(before_ast)
ast.fix_missing_locations(after_ast)
# Assert
ast_fun_assert(after_ast, expected_ast)

AssertionError: Assertion Failed

# Ejercicio 4: Template Method
Dada una clase con un método que llama a metodos de la clase hija, completela declarando los metodos faltantes en la clase para obligar a la clase hija a sobre-escribirlos.

Complete la clase **AddMissingMethodsTransformer**


## Input

In [12]:
# code: ReportMatchineV1
class ReportMachine:
  def export(self):
    self.exportHeader()
    self.exportBody()
    self.exportFooter()

  def exportHeader(self):
    print("This is a really nice report")

## Output

In [13]:
# code: ReportMatchineV2
class ReportMachine:
  def export(self):
    self.exportHeader()
    self.exportBody()
    self.exportFooter()

  def exportHeader(self):
    print("This is a really nice report")

  def exportBody(self):
    pass

  def exportFooter(self):
    pass

## Classes to help you

In [14]:
import ast

class DefinedMethodsVisitor(ast.NodeVisitor):
    def __init__(self):
        self.defined_methods = set()

    def visit_FunctionDef(self, node):
        self.defined_methods.add(node.name)
        self.generic_visit(node)

class MethodCallsVisitor(ast.NodeVisitor):
    def __init__(self):
        self.called_methods = []

    def visit_Attribute(self, node):
        if isinstance(node.value, ast.Name) and node.value.id == 'self':
          if node.attr not in self.called_methods:
            self.called_methods.append(node.attr)
        self.generic_visit(node)

## Complete the trasnformer

In [18]:
class AddMissingMethodsTransformer(ast.NodeTransformer):
  def __init__(self, missing_methods):
        self.missing_methods = missing_methods
  # Completa tu código a continuación


## Method to Test

In [19]:
def transform(class_ast):
  defined_visitor = DefinedMethodsVisitor()
  defined_visitor.visit(class_ast)
  calls_visitor = MethodCallsVisitor()
  calls_visitor.visit(class_ast)

  undefined_methods = []
  for ma in calls_visitor.called_methods:
    if ma not in defined_visitor.defined_methods:
      undefined_methods.append(ma)


  transformer = AddMissingMethodsTransformer(undefined_methods)
  transformed_tree = transformer.visit(class_ast)
  ast.fix_missing_locations(class_ast)
  return transformed_tree


## Test

In [20]:
# Arrange
before_ast =  ast.parse(source_cell("ReportMatchineV1"))
expected_ast =  ast.parse(source_cell("ReportMatchineV2"))

# Act
after_ast = transform(before_ast)
# Assert
ast_cls_assert(after_ast, expected_ast)

Result:
class ReportMachine:

    def export(self):
        self.exportHeader()
        self.exportBody()
        self.exportFooter()

    def exportHeader(self):
        print('This is a really nice report')
Expected:
class ReportMachine:

    def export(self):
        self.exportHeader()
        self.exportBody()
        self.exportFooter()

    def exportHeader(self):
        print('This is a really nice report')

    def exportBody(self):
        pass

    def exportFooter(self):
        pass


AssertionError: Assertion Failed

# Ejercicio 5: Override __str__
Automaticamente sobrescribir el método `__str__`

Complete la clase **StrMethodInjector**


## Input

In [21]:
# code: RectangleV1
class Rectangle():
  def __init__(self):
    self.x = 5
    self.y = 5
    self.width =  100
    self.height =  200

## Output

In [22]:
# code: RectangleV2
class Rectangle():
  def __init__(self):
    self.x = 5
    self.y = 5
    self.width =  100
    self.height =  200
  def __str__(self):
    return f'Rectangle: x={self.x} y={self.y} width={self.width} height={self.height}'

## Classes to help you

In [23]:
import ast

class ClassInspector(ast.NodeVisitor):
    def __init__(self):
        self.methods = []      # To store method names
        self.attributes = []   # To store self attributes
        self.class_name = ""

    def visit_FunctionDef(self, node):
        # Collect the name of every method in the class
        if node.name not in self.methods:
          self.methods.append(node.name)
        self.generic_visit(node)  # Visit the body of the function to find attributes

    def visit_ClassDef(self, node: ClassDef) -> Any:
        self.class_name = node.name
        self.generic_visit(node)

    def visit_Attribute(self, node):
        # Collect attributes accessed through 'self'
        if isinstance(node.value, ast.Name) and node.value.id == 'self':
          if node.attr not in self.attributes:
            self.attributes.append(node.attr)
        self.generic_visit(node)

    def get_non_method_attributes(self):
        # Attributes that are not methods
        attrs =[]
        for a in self.attributes:
          if a not in self.methods:
            attrs.append(a)

        return attrs

    def get_class_name(self):
        return self.class_name


## Complete the trasnformer

In [24]:
class StrMethodInjector(ast.NodeTransformer):
    # Complete
    pass


## Method to Test

In [25]:
def transform2(class_ast):
  inspector = ClassInspector()
  inspector.visit(class_ast)

  transformer = StrMethodInjector(inspector.get_non_method_attributes(), inspector.get_class_name())
  transformed_tree = transformer.visit(class_ast)
  ast.fix_missing_locations(class_ast)
  return transformed_tree



## Test

In [26]:
# Arrange
before_ast =  ast.parse(source_cell("RectangleV1"))
expected_ast =  ast.parse(source_cell("RectangleV2"))
# Act
after_ast = transform2(before_ast)
# Assert
ast_cls_assert(after_ast, expected_ast)

TypeError: StrMethodInjector() takes no arguments